CS524: Introduction to Optimization Lecture 13
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## October 9, 2023
--------------

# The shortest path problem: 

Find the shortest path from node s to node y in the following directed network

![alt text](shortestpath.png "")

In [1]:
%load_ext gams.magic

# Data

In [2]:
%%gams

set N "nodes" /s,u,x,v,z,y/;
alias (N,j);

parameters S(N) "net supplies for nodes" /s 1, y -1/, #supplies for other nodes will be set to 0 by default

         c(N,N) "unit cost on arcs" /
          s.u   1
          s.x   3
          u.s   1
          u.x   3
          u.z   2
          u.v   4
          x.u   3
          x.z   1
          x.y   2
          z.u   3
          z.v   1
          z.y   3
          v.y   2/;
set A(N,N) "arcs";  A(N,j) = yes$(C(N,j) > 0); 

# Instead could generate data using python

In [3]:
gams.reset()
m = gams.exchange_container
N = m.addSet('N',records=['s','u','x','v','z','y'])
j = m.addAlias('j',N)
S = m.addParameter('S',[N],records=[('s', 1), ('y',-1)])
c = m.addParameter('c',[N,N],records=[('s','u', 1), ('s','x', 3), 
    ('u','s', 1), ('u','x',3), ('u','z', 2), ('u','v',4),
    ('x','u', 3), ('x','z',1), ('x','y', 2), ('z','u',3),
    ('z','v', 1), ('z','y',3), ('v','y', 2)])
A = m.addSet('A',[N,N],records=c.records.iloc[:,0:2])

In [4]:
A = m.data['A']
display(A.toList())
c = m.data['c']
display(c.records)

[('s', 'u'),
 ('s', 'x'),
 ('u', 's'),
 ('u', 'x'),
 ('u', 'z'),
 ('u', 'v'),
 ('x', 'u'),
 ('x', 'z'),
 ('x', 'y'),
 ('z', 'u'),
 ('z', 'v'),
 ('z', 'y'),
 ('v', 'y')]

,N_0,N_1,value
0,s,u,1.0
1,s,x,3.0
2,u,s,1.0
3,u,x,3.0
4,u,z,2.0
5,u,v,4.0
6,x,u,3.0
7,x,z,1.0
8,x,y,2.0
9,z,u,3.0


# Model

In [5]:
%%gams

variable totalcost, x(N,N);
positive variable x;

equations balance(N), obj;

balance(N)..
        sum(j$A(N,j), x(N,j)) - sum(j$A(j,N), x(j,N)) =e= s(N);
obj..
        totalcost =e= sum(A, c(A)*x(A));

model sp/balance, obj/;

#Add the following statements to inform GAMS to use the network simplex method to solve your model.
#The $ symbol has to be the first character in the line.

option lp = cplex;
$onecho > cplex.opt
lpmethod 3
$offecho
sp.optfile = 1;


#GAMS option limrow specifies the number of equations to include in GAMS listing file
option limrow = 3;

#GAMS option solprint on/off: whether to not to include the solution report part in the listing file
option solprint = on;

solve sp using lp minimizing totalcost;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),5.0,7,14,LP,CPLEX,0.01


In [6]:
gams.gams_lst('-e')
# gams.gams_lst('')

In [7]:
# gams.gams('display x.l, totalcost.l;')
x = m.data['x']
totalcost = m.data['totalcost']
display(totalcost.records, x.records)
# but following is better
print(f'Total cost will be {totalcost.toValue()}')
display(x.pivot(fill_value=''))

,level,marginal,lower,upper,scale
0,5.0,0.0,-inf,inf,1.0


,N_0,N_1,level,marginal,lower,upper,scale
0,s,u,0.0,1.0,0.0,inf,1.0
1,s,x,1.0,0.0,0.0,inf,1.0
2,u,s,0.0,1.0,0.0,inf,1.0
3,u,x,0.0,0.0,0.0,inf,1.0
4,u,v,0.0,1.0,0.0,inf,1.0
5,u,z,0.0,-0.0,0.0,inf,1.0
6,x,u,0.0,6.0,0.0,inf,1.0
7,x,z,0.0,2.0,0.0,inf,1.0
8,x,y,1.0,0.0,0.0,inf,1.0
9,v,y,0.0,0.0,0.0,inf,1.0


Total cost will be 5.0


,u,x,s,v,z,y
s,0.0,1.0,,,,
u,,0.0,0.0,0.0,0.0,
x,0.0,,,,0.0,1.0
v,,,,,,0.0
z,0.0,,,0.0,,0.0


In [8]:
gams.gams_cleanup('--closedown')